In [24]:
import openpyxl
!pip install numpy==1.24.4 pandas==1.5.3 openpyxl==3.0.10
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter # openpyxlで列番号をアルファベットに変換するために必要
import re #正規表現モジュール使用　日本語と半角を調べる用
from openpyxl.styles import Alignment #中央ぞろえ用

# データのファイルパス
file_path = 'python-excel-kadai2.txt'

# タブ区切りで読み込む
df = pd.read_csv(file_path, sep='\t')

# データフレームを表示
#print(df)

# is_numeric_dtype で数値型かチェックする
if pd.api.types.is_numeric_dtype(df['売上']):
    avarage_sales = df['売上'].mean()
else:
    print("警告: '売上' 列が数値型ではありません。平均計算をスキップします。")
    avarage_sales = None # 平均が計算できない場合はNone


# 業績ランクを算出
# avarage_sales がNoneでないか確認
def rank(sales):
    result = ''
    if avarage_sales is not None:
        if sales >= avarage_sales + 50 :
            result = 'A'
        elif sales == avarage_sales:
            result = 'B'
        else:
            result = 'C'
    else:
        result = 'N/A' # 平均がない場合はランク付けできない
    return result

# avarage_sales が計算できた場合のみ「平均売上」と「業績ランク」を追加
if avarage_sales is not None:
    df['平均売上'] = avarage_sales
    df['業績ランク'] = df['売上'].apply(rank)

# データフレームを表示（確認用）
#print("\n加工後のDataFrame:")
#print(df)

# 一度Excelに出力
output_path = '業績.xlsx'
df.to_excel(output_path, sheet_name='Sheet1', index=False, engine='openpyxl')

# openpyxlで列幅調整
wb = load_workbook(output_path)
ws = wb.active

# 列情報取得
num_cols = len(df.columns)

# 文字幅の見積り関数（日本語考慮）
def get_adjusted_length(value):
    text = str(value)
    # 全角文字（漢字・ひらがな・カタカナなど）は幅2、半角は幅1と仮定
    return sum(2 if re.match(r'[^\x00-\x7F]', ch) else 1 for ch in text)

# 各列に対して最大の文字幅を計算して列幅設定
for col_num in range(1, num_cols + 1):
    max_length = 0
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=col_num, max_col=col_num):
        for cell in row:
            if cell.value:
                max_length = max(max_length, get_adjusted_length(cell.value))
    adjusted_width = (max_length + 2) * 1.2
    ws.column_dimensions[get_column_letter(col_num)].width = adjusted_width

# 中央揃えにしたい列名（売上・平均売上は除外）
exclude_columns = ['売上', '平均売上']

for col_idx, col_name in enumerate(df.columns, 1):
    if col_name in exclude_columns:
        continue  # 除外する列はスキップ

    # 該当列のすべてのセルを中央揃えにする
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
        for cell in row:
            cell.alignment = Alignment(horizontal='center')

# 上書き保存
wb.save(output_path)



